# Caricamento librerie

In [1]:
!pip install numpy pandas scipy matplotlib seaborn bokeh statsmodels scikit-learn utilsforecast


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00


In [2]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
# Hide warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# Handling and processing of Data
# ==============================================================================
import numpy as np
import pandas as pd

# Handling and processing of Data for Date (time)
# ==============================================================================
import datetime

# Plot
# ==============================================================================
import matplotlib.pyplot as plt

# Modelling with Sklearn
# ==============================================================================
from sklearn.linear_model import LinearRegression

# Define the plot size
# ==============================================================================
from pylab import rcParams
rcParams['figure.figsize'] = (18,7)


# **Caso di studio: previsione del benessere giornaliero di un utente**

L’obiettivo di questo progetto è valutare se, attraverso modelli di deep learning per il forecasting, sia possibile prevedere il livello di benessere giornaliero di un utente, rappresentato da un indice sintetico chiamato ***activity index*** , utilizzando le informazioni relative alla durata del sonno e all’andamento dell’attività fisica, come la velocità del passo.

# **Dataset originale**

Il dataset originale è composto da 4 colonne:

* `user_id`: identificatore dell'utente
* `date`: giorno in cui è stato raccolto il dato aggregato
* `data_type`: intero che identifica la tipologia di dato raccolto
* `data_value`: valore associato al dato raccolto

I diversi valori della variabile `data_type` si riferiscono a dati aggregati raccolti giornalmente dal dispositivo di ogni singolo utente e identificano le tipologie di dato in base al seguente schema:

1.  numero totale di passi effettuati
2.  peso (kg)
3.  BMI (kg/m^2)
4.  pressione sanguigna sistolica (mmHg)
5.  velocità dell'onda sfigmica arteriosa (PWV), (m/s)
6.  PWV healthiness (1: bassa, 2: sano, 3: troppo alta)
7.  frequenza cardiaca media (bpm)
8.  frequenza cardiaca minima (bpm)
9.  frequenza cardiaca massima (bpm)
10. durata del sonno (ore)
11. orario in cui l'utente si è messo a letto
12. orario in cui l'utente si è alzato dal letto
13. numero di volte in cui l'utente si è svegliato durante il sonno
14. durata del tempo in cui l'utente si è svegliato durante il sonno (ore)
15. tempo impiegato dall'utente per addormentarsi (ore)
16. tempo impiegato dall'utente per alzarsi dal letto (ore)
17. durata di sonno leggero (ore)
18. durata di sonno REM (ore)
19. durata di sonno profondo (ore)
20. tipo di attività
21. durata dell'attività (secondi)
22. calorie consumate durante l'attività
23. frequenza cardiaca media durante l'attività (bpm)
24. frequenza cardiaca minima durante l'attività (bpm)
25. frequenza cardiaca massima durante l'attività (bpm)
26. velocità dell'andatura dei passi (passi al minuto)
27. velocità dell'andatura a distanza (km all'ora)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
MYDRIVE = 'drive/MyDrive/Datasets/'

In [ ]:
df = pd.read_csv(MYDRIVE + "data/data.csv", usecols=[1, 2, 3, 4])
df.columns = ["user_id", "date_local", "data_type", "value"]
print(df.head())

In [ ]:
df_data_type = pd.read_csv(MYDRIVE + "data/data_type.csv")
print(df_data_type.head)

# **Creazione dataset per la previsione**

E' stato costruito un dataset personalizzato contenente esclusivamente i dati utili all’analisi, con particolare focus sulle variabili legate al sonno e all’attività fisica giornaliera di ciascun utente, in particolare sono state estratte le feature: *sleepduration, bedin, bedout, nbawake, awakeduration, timetosleep, timetowake, remduration, deepduration, activity_type, activity_duration, activity_calories, stepsgaitspeed, distancegaitspeed*.

In [ ]:
# Mappatura data_type → nome feature
feature_map = {
    10: "sleepduration", 11: "bedin", 12: "bedout", 13: "nbawake",
    14: "awakeduration", 15: "timetosleep", 16: "timetowake", 18: "remduration",
    19: "deepduration", 20: "activity_type", 21: "activity_duration",
    22: "activity_calories", 26: "stepsgaitspeed", 27: "distancegaitspeed"
}

# Creazione lista di DataFrame per ciascuna feature
dataset_list = [
    df[df['data_type'] == dtype][['user_id', 'date_local', 'value']]
    .rename(columns={'date_local': 'date', 'value': name})
    for dtype, name in feature_map.items()
]

# Merge progressivo su 'user_id' e 'date'
from functools import reduce

dataset = reduce(lambda left, right: pd.merge(left, right, on=['user_id', 'date'], how='outer'), dataset_list)

# Pulizia finale
dataset.dropna(subset=['sleepduration', 'stepsgaitspeed'], inplace=True)
dataset.set_index(['user_id', 'date'], inplace=True)

# Salvataggio
dataset.to_csv(MYDRIVE + 'data/processed_dataset.csv')


In [ ]:
dataset = pd.read_csv(MYDRIVE + 'data/processed_dataset.csv', index_col=[0, 1])  # Assicura che user_id e date siano indici
dataset

# **Calcolo indice di attività fisica**

La funzione calcola un punteggio (activity_index) per ogni riga del dataset (cioè per ogni giorno di ogni utente), bilanciando:

- Bonus per attività e qualità del sonno, ad esempio chi fa movimento e consuma calorie
- Penalizza poco movimento, poco sonno e risvegli notturni

Questo indice, ottenuto tramite una funzione appositamente definita, combina diverse informazioni relative all’attività fisica e alla qualità del sonno (ad esempio calorie bruciate, durata del sonno e risvegli notturni) in un unico valore numerico utile per la previsione del benessere giornaliero.

In [ ]:
def get_activity_score(row):
    score = 0

    # Calorie bruciate
    if not pd.isna(row['activity_calories']):
        if row['activity_calories'] > 500:
            score += 3
        elif row['activity_calories'] > 300:
            score += 2
        elif row['activity_calories'] > 100:
            score += 1

    # Durata attività fisica
    if not pd.isna(row['activity_duration']):
        duration_min = row['activity_duration'] / 60
        if duration_min > 60:
            score += 3
        elif duration_min > 30:
            score += 2
        elif duration_min > 10:
            score += 1

    # Sleepduration
    if not pd.isna(row['sleepduration']):
        if row['sleepduration'] >= 7:
            score += 2
        elif row['sleepduration'] >= 6:
            score += 1

    # Awakeduration
    if not pd.isna(row['awakeduration']):
        if row['awakeduration'] <= 0.5:
            score += 2
        elif row['awakeduration'] <= 1:
            score += 1

    # Deep sleep
    if not pd.isna(row['deepduration']):
        if row['deepduration'] > 1:
            score += 1

    # REM
    if not pd.isna(row['remduration']):
        if 1 <= row['remduration'] <= 2:
            score += 1

    # Passo veloce
    if not pd.isna(row['stepsgaitspeed']):
        if row['stepsgaitspeed'] > 100:
            score += 1

    # Distanza
    if not pd.isna(row['distancegaitspeed']):
        if row['distancegaitspeed'] > 2:
            score += 1

    return score


La colonna activity_index viene calcolata e aggiunta al dataset.

In [ ]:
dataset['activity_index'] = dataset.apply(get_activity_score, axis=1)


# **Creazione sequenze valide di 15 giorni**

Per la preparazione dei dati è stato implementato un algoritmo che costruisce sequenze temporali di 15 giorni consecutivi (con massimo un giorno di gap) per ciascun utente. Questo passaggio consente di trasformare i dati grezzi in finestre temporali strutturate, fondamentali per permettere al modello Informer di apprendere le dinamiche temporali e fare previsioni sull'andamento futuro dell'activity index, stimando il valore dell’indice di attività del giorno successivo alla sequenza, ovvero il sedicesimo giorno.

In [ ]:
target_sequence_len = 15
sequences = []
target = []

# Resettiamo e convertiamo 'date' in datetime
dataset = dataset.reset_index()
dataset['date'] = pd.to_datetime(dataset['date'])
dataset = dataset.set_index(['user_id', 'date'])

dataset = dataset.sort_index(level=['user_id', 'date'])  # Ordina

current_sequence = []
last_user = None
last_date = None

for (user, date), row in dataset.iterrows():
    if last_user is not None:
        if user == last_user:
            if (date - last_date).days in [1, 2]:
                current_sequence.append(((user, date), row))
                if len(current_sequence) == target_sequence_len:
                    sequences.append(current_sequence.copy())
                    current_sequence.pop(0)
            else:
                current_sequence = [((user, date), row)]
        else:
            current_sequence = [((user, date), row)]
    else:
        current_sequence.append(((user, date), row))

    last_user = user
    last_date = date

print(f"Numero totale di sequenze valide trovate: {len(sequences)}")

In [ ]:
from collections import Counter

# Estrai l'user_id da ogni sequenza
user_ids_in_sequences = [sequence[0][0][0] for sequence in sequences]  # sequence[0][0][0] -> user_id della prima riga della sequenza

# Conta le sequenze per ogni user_id
user_counts = Counter(user_ids_in_sequences)

# Stampa il conteggio
print("Numero sequenze per user_id:", user_counts)

# **Preparazione dataset finale per NeuralForecast**

In questa fase viene costruito il dataset finale utilizzato per l’addestramento del modello Informer. Per ogni sequenza temporale valida, vengono estratte le principali variabili di interesse: passi, durata del sonno e activity index. Successivamente, per ogni sequenza, viene identificata la data del giorno target, ovvero il giorno successivo alla fine della sequenza, e recuperato il relativo valore di activity_index che rappresenta il valore da prevedere (target y). Solo le sequenze per cui è disponibile il valore target vengono mantenute nel dataset finale.
Infine, i dati vengono strutturati secondo il formato richiesto dalla libreria NeuralForecast, rinominando la colonna dell’utente in unique_id e creando un dataframe pronto per l’addestramento del modello.

In [ ]:
final_data = []

for seq in sequences:
    user_id = seq[0][0][0]  # user_id della sequenza
    seq_sorted = sorted(seq, key=lambda x: x[0][1])  # Ordina per data

    # Costruzione del DataFrame della sequenza con anche l'activity_index
    data = []
    for (user, date), row in seq_sorted:
        data.append({
            'user_id': user,
            'date': date,
            'stepsgaitspeed': row['stepsgaitspeed'],
            'sleepduration': row['sleepduration'],
            'activity_index': row['activity_index']
        })
    seq_df = pd.DataFrame(data)

    # Trova la data del giorno target (giorno dopo la fine della sequenza)
    last_date_in_sequence = seq_sorted[-1][0][1]
    target_date = last_date_in_sequence + pd.Timedelta(days=1)

    # Recupero del target (activity_index del giorno successivo)
    try:
        target_row = dataset['activity_index'].loc[(user_id, target_date)]
        target_value = target_row.iloc[0] if isinstance(target_row, pd.Series) else target_row
    except KeyError:
        continue  # Salta la sequenza se non esiste il target

    if pd.isna(target_value):
        continue  # Salta la sequenza se il target è NaN

    seq_df['ds'] = seq_df['date']
    seq_df['y'] = target_value
    final_data.append(seq_df[['user_id', 'ds', 'stepsgaitspeed', 'sleepduration', 'activity_index', 'y']])

# Concatenazione finale
if final_data:
    df_seq = pd.concat(final_data).reset_index(drop=True)
    df_seq.rename(columns={'user_id': 'unique_id'}, inplace=True)
    print("Dataset finale NeuralForecast creato con successo!")
    print(df_seq.head())
else:
    print("Nessuna sequenza completa con target trovata. Dataset vuoto.")


# **Split Train/Test**

Dopo la preparazione del dataset finale, è stato effettuato uno split dei dati in un set di addestramento e un set di test per valutare le performance del modello in fase di previsione. La suddivisione è avvenuta per utente.

In [ ]:
# Controllo che df_seq esista
if 'df_seq' not in locals():
    print("Errore: df_seq non è definito. Esegui prima la preparazione del dataset.")
else:
    from sklearn.model_selection import train_test_split

    # Controllo il dataset
    print("df_seq caricato correttamente!")
    print(df_seq.head())

    # Divido gli utenti unici per evitare leakage
    user_ids = df_seq['unique_id'].unique()
    train_users, test_users = train_test_split(user_ids, test_size=0.3, random_state=42)

    # Costruisco i due dataset
    train_df = df_seq[df_seq['unique_id'].isin(train_users)].reset_index(drop=True)
    test_df = df_seq[df_seq['unique_id'].isin(test_users)].reset_index(drop=True)

    # Stampo le dimensioni
    print(f"Dimensione train: {train_df.shape}")
    print(f"Dimensione test: {test_df.shape}")


In [ ]:
!pip install neuralforecast torchmetrics

# **Definizione del modello Informer utilizzando NeuralForcast**

Per la fase di modellazione è stato utilizzato il framework NeuralForecast, che consente di integrare modelli di deep learning per la previsione di serie temporali multivariate. Nello specifico, è stato implementato il modello Informer.
I parametri utilizzati sono stati gestiti direttamente dalla libreria NeuralForecast, che si occupa di costruire e addestrare l’intera architettura encoder-decoder del modello in modo automatico e ottimizzato per il forecasting.

**Il modello Informer - Breve descrizione e funzionamento dell’architettura**

Nel progetto è stato utilizzato Informer, un modello di deep learning basato su architettura Transformer, progettato per la previsione su serie temporali lunghe e multivariate.

A differenza dei Transformer classici, l’Informer è stato ottimizzato per gestire lunghe sequenze riducendo il carico computazionale e migliorando la capacità predittiva grazie a:

- ProbSparse Attention: seleziona solo le informazioni più rilevanti all’interno della sequenza, riducendo il numero di operazioni necessarie e migliorando l’efficienza sui dati lunghi.
- Attention Distillation: riduce progressivamente la complessità della sequenza filtrando i segnali meno importanti e mantenendo solo le componenti dominanti.
- Decoder efficiente (MLP): consente di generare più previsioni in un solo passaggio, velocizzando la fase di inferenza.
- Embedding temporali e posizionali: catturano sia le dipendenze tra i giorni che la posizione temporale delle osservazioni.

**Funzionamento dell’architettura Informer**

L’architettura dell’Informer si basa su due componenti principali:

- Encoder: riceve la sequenza di input (nel nostro caso 15 giorni di dati su attività e sonno) e applica la ProbSparse Attention e la distillazione per estrarre le dipendenze temporali più rilevanti, riducendo la dimensione della sequenza ma mantenendo l’informazione principale.

- Decoder: utilizza l’output dell’encoder e genera la previsione per l’orizzonte desiderato (nel progetto, il giorno 16). Il decoder è progettato per elaborare in modo efficiente anche lunghe finestre temporali e restituire sia la previsione puntuale che le bande di incertezza.

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import Informer
from neuralforecast.losses.pytorch import DistributionLoss

# Numero di giorni della sequenza passata al modello (15 giorni * 3 variabili)
input_size = target_sequence_len * 3  # stepsgaitspeed + sleepduration + activity_index

model = Informer(
    h=1,  # Previsione a 1 giorno avanti
    input_size=input_size,
    hidden_size=512,
    conv_hidden_size=128,
    n_head=8,
    loss=DistributionLoss(distribution='StudentT', level=[80, 95]),
    scaler_type='standard',
    learning_rate=1e-3,
    encoder_layers=2,
    decoder_layers=2,
    max_steps=500,
    val_check_steps=50,
    windows_batch_size=32,
    batch_size=32,
    # Ottimizzazioni
    dropout=0.2,                   # Aiuta contro l'overfitting
    factor=4,                      # Aumenta la sparsity

)

In [ ]:
# Istanzio NeuralForecast
nf = NeuralForecast(models=[model], freq='D')

# **Addestramento del modello**

In [ ]:
print("Inizio addestramento Informer...")
nf.fit(df=train_df, val_size=5)
print("Addestramento completato!")

# **Previsioni sul test set**

Modelli:

- Informer ➔ la previsione puntuale del modello
- Informer-hi-80 / hi-95 ➔ parte alta dell'intervallo di predizione (80% - 95%)
- Informer-lo-80 / lo-95 ➔ parte bassa dell'intervallo di predizione
- Informer-median ➔ la mediana della distribuzione predetta

In [ ]:
forecast = nf.predict(df=test_df)
print("Previsione completata!")
forecast.head()

In [ ]:
print(f"Forecast shape: {forecast.shape}")
print(forecast.head())

# Assicuro i tipi corretti per il merge
forecast['unique_id'] = forecast['unique_id'].astype(str)
forecast['ds'] = pd.to_datetime(forecast['ds'])
dataset = dataset.reset_index()
dataset['user_id'] = dataset['user_id'].astype(str)
dataset['date'] = pd.to_datetime(dataset['date'])
dataset = dataset.set_index(['user_id', 'date'])

# Creazione della lista dei target reali basandomi sulle date previste
true_targets = []
for idx, row in forecast.iterrows():
    try:
        y_real = dataset.loc[(row['unique_id'], row['ds']), 'activity_index']
        # Se ci sono più valori, prendere il primo
        y_real = y_real.iloc[0] if isinstance(y_real, pd.Series) else y_real
        true_targets.append({'unique_id': row['unique_id'], 'ds': row['ds'], 'y_real': y_real})
    except KeyError:
        # Se non esiste la coppia, mettere NaN
        true_targets.append({'unique_id': row['unique_id'], 'ds': row['ds'], 'y_real': np.nan})

# Costruzione del DataFrame dei target reali
real_targets_df = pd.DataFrame(true_targets)
print("\nTarget recuperati:")
print(real_targets_df.head())

# **Valutazione Classica sul test**

In questa fase è stata effettuata la valutazione quantitativa del modello. Le previsioni generate dal modello Informer sono state unite ai valori reali dell’activity index recuperati dal dataset tramite un'operazione di merge sulla chiave composta da utente (unique_id) e data (ds).
Prima del calcolo delle metriche, sono stati eliminati eventuali record con valori mancanti per garantire la correttezza delle stime. La valutazione è stata condotta solo sulle righe con dati completi.

In [ ]:
# Merge tra forecast e target reali
merged = pd.merge(forecast, real_targets_df, on=['unique_id', 'ds'], how='inner')

# Rimozione di eventuali NaN prima del calcolo
merged = merged.dropna(subset=['y_real', 'Informer'])
print(f"\nRighe valide per il calcolo: {merged.shape[0]}")

# Calcolo delle metriche solo se ci sono righe valide
if merged.empty:
    print("Nessuna riga valida per calcolare le metriche.")
else:
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    mse = mean_squared_error(merged['y_real'], merged['Informer'])
    mae = mean_absolute_error(merged['y_real'], merged['Informer'])
    rmse = np.sqrt(mse)
    r2 = r2_score(merged['y_real'], merged['Informer'])

    print("\nRISULTATI MODELLO INFORMER")
    print(f"MSE:  {mse:.3f}")
    print(f"MAE:  {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²:   {r2:.3f}")


# **Valutazione del modello - CrossValidation**

Oltre alla valutazione delle prestazioni su test set, è stata eseguita una cross-validation temporale per simulare il comportamento del modello durante l’addestramento e valutare la sua stabilità nel tempo.

La cross-validation è stata effettuata con 10 finestre (n_windows=10), ciascuna rappresentante uno scenario di addestramento e validazione su porzioni differenti della serie temporale. Per ogni finestra è stato calcolato l’errore assoluto medio (MAE) come metrica di riferimento.

In [ ]:
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error

#Eseguo la cross-validation
cv_result = nf.cross_validation(
    df=train_df,
    n_windows=10  # Numero di split da effettuare
)

print("Cross-validation completata!")
print(cv_result.head())

In [ ]:
# Calcolo MAE per ogni finestra della cross-validation
epoch_mae = cv_result.groupby('cutoff').apply(
    lambda g: mae(g['y'], g['Informer'])
).reset_index(name='mae_loss')

# Applicazione dello smoothing alla MAE
epoch_mae['mae_smooth'] = epoch_mae['mae_loss'].rolling(window=3, min_periods=1).mean()

# Plot MAE smussata
plt.figure(figsize=(12, 6))
plt.plot(epoch_mae['mae_smooth'], color='blue', label='Smoothed Validation MAE')
plt.fill_between(epoch_mae.index, epoch_mae['mae_smooth'], alpha=0.2, color='blue')

# Trovo il minimo
min_idx = epoch_mae['mae_smooth'].idxmin()
min_val = epoch_mae['mae_smooth'].min()
plt.scatter(min_idx, min_val, color='red', label=f'Min MAE: {min_val:.2f}')

plt.title('Smoothed MAE Curve - Cross-Validation Performance per Epoch')
plt.xlabel('Epochs / CV Windows')
plt.ylabel('MAE Loss')
plt.ylim(0, epoch_mae['mae_smooth'].max() + 2)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('mae_crossval_plot.png', dpi=300)
plt.show()

# **Personalized Fitness Recommender System**

Sulla base delle predizioni fornite dal modello Informer per l'activity_index del giorno successivo, è stato integrato un sistema di raccomandazione personalizzato per ciascun utente, con l'obiettivo di migliorare il benessere attraverso suggerimenti mirati su sonno e attività fisica, come ad esempio:

- Dormi più a lungo
- Fai più attività fisica
- Riduci i risvegli notturni

In [ ]:
def detailed_fitness_recommendation(row):
    # Restituisce una raccomandazione personalizzata basata sull'activity_index e i fattori deboli.
    index = row['activity_index']
    feedback = []

    if pd.isna(row['sleepduration']) or row['sleepduration'] < 6:
        feedback.append("dormi di pi\u00f9")
    if pd.isna(row['activity_duration']) or row['activity_duration'] / 60 < 30:
        feedback.append("fai pi\u00f9 attivit\u00e0 fisica")
    if pd.isna(row['awakeduration']) or row['awakeduration'] > 1:
        feedback.append("riduci i risvegli notturni")
    if pd.isna(row['deepduration']) or row['deepduration'] < 1:
        feedback.append("aumenta il sonno profondo")
    if pd.isna(row['remduration']) or not (1 <= row['remduration'] <= 2):
        feedback.append("ottimizza la fase REM")
    if pd.isna(row['stepsgaitspeed']) or row['stepsgaitspeed'] <= 100:
        feedback.append("cammina a passo pi\u00f9 sostenuto")
    if pd.isna(row['distancegaitspeed']) or row['distancegaitspeed'] <= 2:
        feedback.append("aumenta la distanza percorsa")
    if pd.isna(row['activity_calories']) or row['activity_calories'] <= 100:
        feedback.append("brucia pi\u00f9 calorie")

    # Messaggio base in base all'index
    if index >= 8:
        base_msg = "Ottimo lavoro! Continua cos\u00ec."
    elif 6 <= index < 8:
        base_msg = "Buon livello, ma c'\u00e8 margine di miglioramento."
    elif 4 <= index < 6:
        base_msg = "Livello moderato. Attenzione:"
    else:
        base_msg = "Livello basso. Consigliato intervenire su:"

    if feedback:
        return f"{base_msg} {', '.join(feedback[:3])}."
    else:
        return base_msg

Per generare raccomandazioni personalizzate, le previsioni dell'activity index sono state arricchite con le feature originali del dataset tramite un'operazione di merge sui campi user_id e data. Questo ha permesso di associare a ciascuna previsione informazioni reali come la durata del sonno o le calorie bruciate, fondamentali per applicare una funzione di raccomandazione dettagliata e fornire suggerimenti specifici per il benessere dell’utente.

In [ ]:
# Unione forecast + dati reali per avere accesso alle feature necessarie
forecast_with_features = pd.merge(
    forecast,
    dataset.reset_index(),
    left_on=['unique_id', 'ds'],
    right_on=['user_id', 'date'],
    how='left'
)

# Applicazione della funzione di raccomandazione dettagliata
forecast_with_features['recommendation'] = forecast_with_features.apply(detailed_fitness_recommendation, axis=1)

In [ ]:
#Visualizzazione di alcune raccomandazioni
import random
sample_users = random.sample(list(forecast_with_features['unique_id'].unique()), 5)

for user in sample_users:
    user_data = forecast_with_features[forecast_with_features['unique_id'] == user].sort_values('ds').tail(1)
    print(f"\nUtente: {user}")
    print(f"Data previsione: {user_data['ds'].dt.strftime('%Y-%m-%d').values[0]}")
    print(f"Activity Index previsto: {user_data['Informer'].values[0]:.2f}")
    print(f"Raccomandazione: {user_data['recommendation'].values[0]}")

# **Creazione dei modelli per confronto con Informer e Valutazione**

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import DistributionLoss
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def run_model(model_class, model_name, target_col='activity_index'):
    print(f"\nAvvio addestramento del modello {model_name}...")

    # Inizializzazione del modello
    model = model_class(
        h=1,
        input_size=target_sequence_len * 3,
        learning_rate=1e-3,
        max_steps=500,
        val_check_steps=50,
        windows_batch_size=32,
        batch_size=32,
        loss=DistributionLoss(distribution='StudentT', level=[80, 95]),
        early_stop_patience_steps=150,
        num_lr_decays=2,
        scaler_type='standard'
    )

    nf = NeuralForecast(models=[model], freq='D')
    nf.fit(df=train_df, val_size=1)

    forecast = nf.predict(df=test_df)
    print("Previsione completata!")

    # Conversioni e allineamento date/user
    forecast['unique_id'] = forecast['unique_id'].astype(str)
    forecast['ds'] = pd.to_datetime(forecast['ds'])

    dataset_copy = dataset.reset_index()
    dataset_copy['user_id'] = dataset_copy['user_id'].astype(str)
    dataset_copy['date'] = pd.to_datetime(dataset_copy['date'])
    dataset_copy = dataset_copy.set_index(['user_id', 'date'])

    # Recupero dei target reali
    true_targets = []
    for idx, row in forecast.iterrows():
        try:
            y_real = dataset_copy.loc[(row['unique_id'], row['ds']), target_col]
            y_real = y_real.iloc[0] if isinstance(y_real, pd.Series) else y_real
        except KeyError:
            y_real = np.nan
        true_targets.append({'unique_id': row['unique_id'], 'ds': row['ds'], 'y_real': y_real})

    real_targets_df = pd.DataFrame(true_targets)

    # Merge tra predizione e valore reale
    merged = pd.merge(forecast, real_targets_df, on=['unique_id', 'ds'], how='inner')
    merged = merged.dropna(subset=['y_real', model_name])
    print(f"\nRighe valide per il calcolo delle metriche: {merged.shape[0]}")

    # Metriche
    if merged.empty:
        print("Nessuna riga valida per il calcolo.")
    else:
        mse = mean_squared_error(merged['y_real'], merged[model_name])
        mae = mean_absolute_error(merged['y_real'], merged[model_name])
        rmse = np.sqrt(mse)
        r2 = r2_score(merged['y_real'], merged[model_name])

        print(f"\n RISULTATI MODELLO {model_name}")
        print(f"MSE:  {mse:.3f}")
        print(f"MAE:  {mae:.3f}")
        print(f"RMSE: {rmse:.3f}")
        print(f"R²:   {r2:.3f}")

    return merged


# **modello PatchTST**
Il PatchTST è un modello di deep learning di ultima generazione, progettato specificamente per il forecasting di serie temporali multivariate. A differenza dei modelli tradizionali, PatchTST suddivide le sequenze temporali in piccoli segmenti (patch), facilitando l’apprendimento di pattern locali e globali. Questa struttura lo rende particolarmente efficace su dataset complessi come quello utilizzato in questo progetto, dove sono presenti più variabili correlate (sonno, attività fisica, indice di attività). PatchTST sfrutta i punti di forza dei Transformer, migliorando la gestione di lunghe sequenze e riducendo il rischio di overfitting

In [ ]:
from neuralforecast.models import PatchTST
merged_patchtst = run_model(PatchTST, 'PatchTST')


# **modello Autoformer**
Autoformer è un modello avanzato basato su Transformer, progettato per il forecasting di serie temporali a lungo termine. La sua architettura introduce un meccanismo di autocorrelazione che consente di catturare le dipendenze periodiche nei dati in modo più efficiente. A differenza dei Transformer tradizionali, Autoformer elimina la necessità di posizioni assolute, concentrandosi invece sulla ripetitività intrinseca delle serie temporali. Questo approccio lo rende particolarmente adatto a scenari in cui i pattern ciclici, come quelli legati al sonno e all’attività fisica, giocano un ruolo centrale nella previsione dell’indice di benessere.

In [ ]:
from neuralforecast.models import Autoformer
merged_autoformer = run_model(Autoformer, 'Autoformer')

# **modello NHITS**
NHITS (Neural Hierarchical Interpolation for Time Series) è un modello MLP-based che adotta un approccio gerarchico per la ricostruzione delle serie temporali. Utilizza una struttura a blocchi progressivi per approssimare il segnale target attraverso raffinamenti successivi, riducendo l’errore di previsione a ogni passaggio. NHITS si distingue per l'efficienza computazionale e la robustezza alle fluttuazioni dei dati.

In [ ]:
from neuralforecast.models import NHITS
merged_nhits = run_model(NHITS, 'NHITS')

# **modello TimesNet**
TimesNet è un modello basato su convoluzioni gerarchiche pensato per il forecasting di serie temporali univariate e multivariate. Introduce un nuovo paradigma che sfrutta kernel convoluzionali a diversi livelli per catturare dinamiche temporali a scala multipla. Grazie alla sua capacità di apprendere rappresentazioni temporali ricche ed efficienti, TimesNet si adatta bene a serie complesse e rumorose.

In [ ]:
from neuralforecast.models import TimesNet
merged_nhits = run_model(TimesNet, 'TimesNet')

# **Conclusione**


Dal confronto finale tra i modelli testati emerge che Informer rappresenta la soluzione più efficace per la previsione dell’activity_index, grazie a prestazioni complessive superiori su tutte le metriche considerate. In particolare, Informer ottiene il MSE più basso (0.923), il MAE più contenuto (0.702) e il valore più alto di R² (0.555), evidenziando una maggiore precisione e una migliore capacità di spiegare la variabilità del target rispetto agli altri modelli.

A confronto, PatchTST mostra performance solide (MSE = 1.341, MAE = 0.880, R² = 0.354), ma meno efficaci nel catturare le dinamiche temporali complesse del dataset. Anche modelli più recenti come TimesNet (MSE = 1.906, MAE = 0.989, R² = 0.082) e Autoformer (MSE = 1.983, MAE = 1.006) non riescono a raggiungere gli stessi livelli di accuratezza. Infine, NHITS risulta il meno performante tra quelli testati, con un MSE di 2.739 e un MAE di 1.153.

Nel complesso, l’architettura di Informer si dimostra particolarmente adatta al contesto multivariato e alle forti dipendenze temporali del problema analizzato, confermandosi come il modello più robusto e affidabile tra quelli sperimentati.